In [ ]:
using PMP, CSV, DataFrames, Distributions, Gadfly, Statistics, Cairo, Fontconfig, Dates

In [ ]:
function qqplot(y::Vector, pd_fit, title)
    y = sort(y)
    n = length(y)
    
    i = 1:n
    xp = i/(n+1)
    
    x = quantile.(pd_fit, xp)  # model quantile
    
    plot(x=x, y=y, Geom.point, Geom.abline(color="red", style=:dash),
        style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
        Guide.ylabel("Empirical quantile"), Guide.xlabel("Model quantile"), Guide.title(title))
end

In [ ]:
function ppplot(y::Vector, pd_fit, title)
    y = sort(y)
    n = length(y)
    
    i = 1:n
    xp = i/(n+1)
    
    yp = cdf.(pd_fit, y)
    
    plot(x=xp, y=yp, Geom.point, Geom.abline(color="red", style=:dash),
        style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
        Guide.ylabel("Model probability"), Guide.xlabel("Empirical probability"), Guide.title(title))
end

---
# Fit on observed precipitation datasets
---
## International Pierre-Elliott-Trudeau Airport of Montréal

In [ ]:
mtl_intla = CSV.read("rain-mtl.csv", DataFrame)
recent = CSV.read("rain_mtl_recent.csv", DataFrame)
append!(mtl_intla, recent);

In [ ]:
hist_mtl = plot(x=mtl_intla.Rain, Geom.histogram(bincount=50), 
            Guide.title("Observed precipitations at Montréal Pierre-Elliott-Trudeau station"),
            Guide.xlabel("Precipitations in mm"))
draw(PNG("figures-03/hist_mtl.png"), hist_mtl)
hist_mtl

In [ ]:
length(mtl_intla.Rain)

In [ ]:
maximum(mtl_intla.Rain)

### 3 parameters Pearson de type I
#### Mixed

In [ ]:
fit_21 = getinitialvalues(PearsonType1b, mtl_intla.Rain)

In [ ]:
qq21 = qqplot(mtl_intla.Rain, PearsonType1b(fit_21...), "Mixed");

In [ ]:
pp21 = ppplot(mtl_intla.Rain, PearsonType1b(fit_21...), "Mixed");

#### MM

In [ ]:
fit_22 = fit_mme(PearsonType1b, mtl_intla.Rain)

In [ ]:
qq22 = qqplot(mtl_intla.Rain, fit_22, "MM");

In [ ]:
pp22 = ppplot(mtl_intla.Rain, fit_22, "MM");

#### Gibbs

In [ ]:
fit_23 = fit_bayes_MH(PearsonType1b, mtl_intla.Rain)

In [ ]:
df23_mtl = DataFrame([:b=>fit_23[1], :α=>fit_23[2], :β=>fit_23[3]]);

In [ ]:
d_23 = PearsonType1b(mean(df23_mtl.b), mean(df23_mtl.α), mean(df23_mtl.β))

In [ ]:
l1=layer(x=mtl_intla.Rain, Geom.histogram(bincount=75, density=true))
l2=layer(x->pdf(d_23, x), 0, 85, color=["Gibbs"])
l3=layer(x->pdf(PearsonType1b(fit_21...), x), 0, 85, color=["Mixed"])
l4=layer(x->pdf(fit_22, x), 0, 85, color=["MM"])
plot(l2, l3, l4, l1, Coord.cartesian(xmin=66, xmax=84, ymax=0.0005), Guide.xlabel("Precipitations in mm"),
    Scale.color_discrete_manual(colorant"yellow3",colorant"palevioletred2",
                                              colorant"mediumpurple1"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, 
                  key_title_font_size=16pt, key_label_font_size=14pt, line_width=1.5pt),
    Guide.colorkey("Method"), Guide.ylabel(""))

In [ ]:
hist1 = plot(yintercept=[mean(df23_mtl.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_mtl.b, Geom.histogram(bincount=20, orientation=:horizontal)), 
    Guide.ylabel("b"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist2 = plot(yintercept=[mean(df23_mtl.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_mtl.α, Geom.histogram(bincount=20, orientation=:horizontal), color=[colorant"yellow3"]), 
    Guide.ylabel("α"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist3 = plot(yintercept=[mean(df23_mtl.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_mtl.β, Geom.histogram(bincount=20, orientation=:horizontal),color=[colorant"palevioletred2"]), 
    Guide.ylabel("β"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))

In [ ]:
trace1 = plot(yintercept=[mean(df23_mtl.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_mtl.b, Geom.line), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace2 = plot(yintercept=[mean(df23_mtl.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_mtl.α, Geom.line, color=[colorant"yellow3"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace3 = plot(yintercept=[mean(df23_mtl.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_mtl.β, Geom.line, color=[colorant"palevioletred2"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))

trace_mtl = title(gridstack([hist1 trace1; hist2 trace2; hist3 trace3]), "Model with a=0")

In [ ]:
qq23 = qqplot(mtl_intla.Rain, d_23, "Gibbs")
draw(PNG("figures-09/qq23.png", 6inch, 6inch), qq23)

pp23 = ppplot(mtl_intla.Rain, d_23, "Gibbs")
draw(PNG("figures-09/pp23.png", 6inch, 6inch), pp23)

In [ ]:
qq_2 = title(gridstack([qq22 qq21 qq23; pp22 pp21 pp23]), "Model with a=0")

### 4 parameters Pearson de type I with a = 0.2 fixed
#### Mixed

In [ ]:
fit_31 = getinitialvalues(PearsonType1, mtl_intla.Rain, 0.2)

In [ ]:
qq31 = qqplot(mtl_intla.Rain, PearsonType1(fit_31...), "Mixed")

pp31 = ppplot(mtl_intla.Rain, PearsonType1(fit_31...), "Mixed")

#### MM

In [ ]:
fit_32 = fit_mme(PearsonType1, mtl_intla.Rain, 0.2)

In [ ]:
qq32 = qqplot(mtl_intla.Rain, fit_32, "MM")

pp32 = ppplot(mtl_intla.Rain, fit_32, "MM")

#### Gibbs

In [ ]:
fit_33 = fit_bayes_MH(PearsonType1, mtl_intla.Rain, 0.199)

In [ ]:
df33_mtl = DataFrame([:b=>fit_33[1], :α=>fit_33[2], :β=>fit_33[3]]);

In [ ]:
d_33 = PearsonType1(0.2, mean(df33_mtl.b), mean(df33_mtl.α), mean(df33_mtl.β))

In [ ]:
hist12 = plot(yintercept=[mean(df33_mtl.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_mtl.b, Geom.histogram(bincount=20, orientation=:horizontal)), Guide.ylabel("b"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist22 = plot(yintercept=[mean(df33_mtl.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_mtl.α, Geom.histogram(bincount=20, orientation=:horizontal), color=[colorant"yellow3"]), 
    Guide.ylabel("α"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist32 = plot(yintercept=[mean(df33_mtl.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_mtl.β, Geom.histogram(bincount=20, orientation=:horizontal),color=[colorant"palevioletred2"]), 
    Guide.ylabel("β"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))

In [ ]:
trace12 = plot(yintercept=[mean(df33_mtl.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_mtl.b, Geom.line), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace22 = plot(yintercept=[mean(df33_mtl.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_mtl.α, Geom.line, color=[colorant"yellow3"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace32 = plot(yintercept=[mean(df33_mtl.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_mtl.β, Geom.line, color=[colorant"palevioletred2"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))

trace_mtl2 = title(gridstack([hist12 trace12; hist22 trace22; hist32 trace32]), "Model with a=0.2")

In [ ]:
qq33 = qqplot(mtl_intla.Rain, d_33, "Gibbs")

pp33 = ppplot(mtl_intla.Rain, d_33, "Gibbs")

In [ ]:
qq_3 = title(gridstack([qq32 qq31 qq33; pp32 pp31 pp33]), "Modèle avec a=0.2")


---
## St-Hubert

In [ ]:
sh_data = CSV.read("rain-sh.csv", DataFrame);

In [ ]:
hist_sh = plot(x=sh_data.Rain, Geom.histogram(bincount=500), 
            Guide.title("Observed precipitations at Saint-Hubert station"),
            Guide.xlabel("Precipitations in mm"))

In [ ]:
length(sh_data.Rain)

In [ ]:
maximum(sh_data.Rain)

### 3 parameters Pearson de type I
#### Mixed

In [ ]:
fit_21 = getinitialvalues(PearsonType1b, sh_data.Rain)

In [ ]:
qq21 = qqplot(sh_data.Rain, PearsonType1b(fit_21...), "Mixed");

pp21 = ppplot(sh_data.Rain, PearsonType1b(fit_21...), "Mixed");

#### MM

In [ ]:
fit_22 = fit_mme(PearsonType1b, sh_data.Rain)

In [ ]:
qq22 = qqplot(sh_data.Rain, fit_22, "MM");

pp22 = ppplot(sh_data.Rain, fit_22, "MM");

#### Gibbs

In [ ]:
fit_23 = fit_bayes_MH(PearsonType1b, sh_data.Rain)

In [ ]:
df23_sh = DataFrame([:b=>fit_23[1], :α=>fit_23[2], :β=>fit_23[3]]);

In [ ]:
d23 = PearsonType1b(mean(df23_sh.b), mean(df23_sh.α), mean(df23_sh.β))

In [ ]:
hist13 = plot(yintercept=[mean(df23_sh.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_sh.b, Geom.histogram(bincount=20, orientation=:horizontal)), Guide.ylabel("b"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist23 = plot(yintercept=[mean(df23_sh.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_sh.α, Geom.histogram(bincount=20, orientation=:horizontal), color=[colorant"yellow3"]), 
    Guide.ylabel("α"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist33 = plot(yintercept=[mean(df23_sh.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_sh.β, Geom.histogram(bincount=20, orientation=:horizontal),color=[colorant"palevioletred2"]), 
    Guide.ylabel("β"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))

In [ ]:
trace13 = plot(yintercept=[mean(df23_sh.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_sh.b, Geom.line), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace23 = plot(yintercept=[mean(df23_sh.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_sh.α, Geom.line, color=[colorant"yellow3"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace33 = plot(yintercept=[mean(df23_sh.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df23_sh.β, Geom.line, color=[colorant"palevioletred2"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))

trace_sh = title(gridstack([hist13 trace13; hist23 trace23; hist33 trace33]), "Model with a=0")

In [ ]:
qq23 = qqplot(sh_data.Rain, d23, "Gibbs");

pp23 = ppplot(sh_data.Rain, d23, "Gibbs");

In [ ]:
qq_2 = title(gridstack([qq22 qq21 qq23; pp22 pp21 pp23]), "Model with a=0")

### 4 parameters Pearson de type with a = 0.2 fixed
#### Mixed

In [ ]:
fit_31 = getinitialvalues(PearsonType1, sh_data.Rain, 0.2)

In [ ]:
qq31 = qqplot(sh_data.Rain, PearsonType1(fit_31...), "Mixed");

pp31 = ppplot(sh_data.Rain, PearsonType1(fit_31...), "Mixed");

#### MM

In [ ]:
fit_32 = fit_mme(PearsonType1, sh_data.Rain, 0.2)

In [ ]:
qq32 = qqplot(sh_data.Rain, fit_32, "MM");

pp32 = ppplot(sh_data.Rain, fit_32, "MM");

#### Gibbs

In [ ]:
fit_33 = fit_bayes_MH(PearsonType1, sh_data.Rain, 0.199)

In [ ]:
df33_sh = DataFrame([:b=>fit_33[1], :α=>fit_33[2], :β=>fit_33[3]]);

In [ ]:
d33 = PearsonType1(0.2, mean(df33_sh.b), mean(df33_sh.α), mean(df33_sh.β))

In [ ]:
hist14 = plot(yintercept=[mean(df33_sh.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_sh.b, Geom.histogram(bincount=20, orientation=:horizontal)), Guide.ylabel("b"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist24 = plot(yintercept=[mean(df33_sh.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_sh.α, Geom.histogram(bincount=20, orientation=:horizontal), color=[colorant"yellow3"]), 
    Guide.ylabel("α"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))
hist34 = plot(yintercept=[mean(df33_sh.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_sh.β, Geom.histogram(bincount=20, orientation=:horizontal),color=[colorant"palevioletred2"]), 
    Guide.ylabel("β"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt))

In [ ]:
trace14 = plot(yintercept=[mean(df33_sh.b)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_sh.b, Geom.line), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace24 = plot(yintercept=[mean(df33_sh.α)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_sh.α, Geom.line, color=[colorant"yellow3"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))
trace34 = plot(yintercept=[mean(df33_sh.β)], Geom.hline(style=:dot, color=[colorant"chartreuse4"]),
    layer(y=df33_sh.β, Geom.line, color=[colorant"palevioletred2"]), Coord.cartesian(xmax=1550),
    style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt),
    Guide.ylabel(" "))

trace_sh2 = title(gridstack([hist14 trace14; hist24 trace24; hist34 trace34]), "Model with a=0.2")

In [ ]:
qq33 = qqplot(sh_data.Rain, d33, "Gibbs");

pp33 = ppplot(sh_data.Rain, d33, "Gibbs");

In [ ]:
qq_3 = title(gridstack([qq32 qq31 qq33; pp32 pp31 pp33]), "Model with a=0.2")